#            DIABETICS PREDICTION USING LOGISTIC REGRESSION 
                        PREDICTING WHETHER A PERSON HAS DIABETES OR NOT 
	
  

# Framing the Problem

The objective is to Predict whether a person has diabetes or not based on a number of labels or features: 
this includes: 
        
        Pregnancy : number of times pregnant
	        
        Glucose: glucose level
		
        Blood Pressure:  Diastolic blood pressure (mm Hg)
		
        Skin Thickness:  Triceps skin fold thickness (mm)
		
        Insulin: 2-Hour serum insulin (mu U/ml)
		
        BMI: Body Mass Index
		
        Diabetes Pedigree Function (DBF): A function that scores the likehood of diabetes based on family history
		
        Age: individual Age (years)
		
        Outcome: 0 = healthy individual, 1 = Diabetic individual
This will involve the use of a supervised learning model (linear regression e.t.c). 

It is a regression task as our objective is to predict the outcome label ( 1 or 0).

Furthermore, it will use Batch Learning as it the model will be trained on all our current available data.

# Getting the Data

we will be using a kaggle dataset that was madee available by National Institute of Diabetes 

and Digestive and Kidney Diseases” as part of the Pima Indians Diabetes Database.

steps to be done:

          (a) Convert the data to a format you can easily manipulate (ex: changing to all numerical)
		  (b) Check the size and type of data (time series, sample).
		  (C) Sample a test set, put it aside. 

Data is all Numerical - so we can move to the next step

# Explore the data: 

in this step we will try to get meaningful insight into our data any correlation between attributes 

steps to be done: 

          (a): Study each attribute and its characteristics ( this includes the Name, Type(int/ float, text, structured)
		  and identify correlation between attributes 
          
		  (b): check the % of missing values in data

		  (c): we Study how we would solve this problem manually.
		  (d): Identify extra data that would be useful (maybe in improving model performance)

		  (e): we are going to outline the usefulness of the task/steps done above
		  (f): Document what you have learned from Data exploration

# DATA PREPARATION:
	In this step we get the data ready for machine learning 
    
    steps to be done: 
	
    1. Data cleaning:
		Fix or remove outliers (optional).
		Fill in missing values (e.g., with zero, mean, median…) or drop their rows (or
		columns)
	2. Feature selection (optional):
		Drop the attributes that provide no useful information for the task.

	3. Feature engineering, where appropriate:
		Discretize continuous features.

		Decompose features (e.g., categorical, date/time, etc.).
		Add promising transformations of features (e.g., log(x), sqrt(x), x^2, etc.).
		Aggregate features into promising new features.

	4. Feature scaling:
		Standardize or normalize features.

# Shortlisting Promising Models

Steps to be done:

	(1). splitting up out training and test data and applying various models

	(2). Performance Measure
		For Perfomance measure it depends on the data - but we consider both
		(a): MAE (Mean absolute Error)
		(b): RMSE ( Root MEan square Error)
	(3). Analyze the most significant variables for each algorithm.

	(4). Analyze the types of errors the models make and What data would a human have used to avoid these errors?

	(5). Perform a quick round of feature selection and engineering.

	(6). Perform one or two more quick iterations of the five previous steps.

	(7). Shortlist the top three to five most promising models, preferring models that
	make different types of errors.

# FINE TUNING SYSTEM
	


# PRESENT SOLUTION:

	In this step we are going to document what we have done, describing what worked and what did not, and system limitations